d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# N-grams Lab

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lab you:<br>
* Learn what n-grams are
* Generate n-grams for each review

In [0]:
%run ../Includes/Classroom-Setup

Preparing the learning environment...

Defining courseware-specific utility methods...

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

The following variables were defined for you: 
 username : melburne42@gmail.com 
 No additional information was provided. 
 
 userhome : dbfs:/user/melburne42@gmail.com 
 No additional information was provided. 
 
 workingDir : dbfs:/user/melburne42@gmail.com/nlp/nlp_02l_n_grams_lab 
 No additional information was provided. 
 All done!

Another commonly used preprocessing step is the generation of n-grams, an ordered sequence of `n` tokens. This can be important when there are meaningful phrases that are made up of multiple words in a specific order. For example knowing that "really good" occurred is more important than just knowing that the tokens "really" and "good" appeared in the text. Sometimes using `n > 2` may be helpful such as extracting the phrase "really highly recommend."

Here is an example of the n-grams with n = 1, 2, and 3 for a simple sentence:

![](https://files.training.databricks.com/images/trigram-updated.png)

In addition to using n-grams on a word level, often times they are also used on a character level so that subwords can be learned and typos can be dealt with.

**Note:** n-grams with `n = 1` is the same as the original tokens list.

Load in our tokenized and processed DataFrame.

In [0]:
processedDF = spark.read.parquet("/mnt/training/reviews/tfidf.parquet")

One implementation of n-grams is SparkML's built-in `NGram` <a href="https://spark.apache.org/docs/latest/ml-features.html#n-gram" target="_blank">function</a>,  which takes an integer `n` and a list of tokens, and creates all possible groups of **exactly** `n` consecutive tokens.

Using SparkML's `NGram` function, fill in the following cell to return a new DataFrame, `ngramDF`, with the column, `ngrams`, containing all n-grams **up to and including** `n = 3`. In other words, append a single column which contains all the tokens (cleaned), bigrams, and trigrams of each corresponding row.


**Note:** We want to construct our n-grams using the `CleanTokens` column, *not* the raw `Tokens` column.

**Hint:** You may want to take a look at <a href = "http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.concat" target="_blank"?> pyspark.sql.functions.concat</a>.

In [0]:
# TODO
# Apply n-grams to processed DataFrame
from pyspark.ml.feature import NGram
from pyspark.sql.functions import concat, col

ngramDF = processedDF.select("Text", "Tokens", "CleanTokens")

# Create the ngram2 and ngram3 transformers
ngram2 = NGram(n=2, inputCol="CleanTokens", outputCol="ngrams2")
ngram3 = NGram(n=3, inputCol="CleanTokens", outputCol="ngrams3")

# Perform the transformation
ngramDF = ngram2.transform(ngramDF)
ngramDF = ngram3.transform(ngramDF)

# Combine tokens, bigrams, and trigrams
ngramDF = ngramDF.withColumn(
    "ngrams", concat(col("tokens"), col("ngrams2"), col("ngrams3"))
)

display(ngramDF.select("ngrams"))

ngrams
"List(my, little, westie, benji, was, having, heartburn, diarrhea, and, upset, stomach, issues, for, quite, a, few, months, he, would, throw, up, almost, daily, we, took, him, to, an, internest, who, recommended, we, feed, him, a, kibble, containing, only, two, ingredients, before, we, proceed, with, other, tests, this, is, one, of, the, kibbles, she, suggested, it, was, in, two, days, that, his, vomiting, and, diarrhia, stopped, i, highly, recommend, this, product, thank, you, natural, balance, little westie, westie benji, benji heartburn, heartburn diarrhea, diarrhea upset, upset stomach, stomach issues, issues quite, quite months, months throw, throw almost, almost daily, daily took, took internest, internest recommended, recommended feed, feed kibble, kibble containing, containing two, two ingredients, ingredients proceed, proceed tests, tests one, one kibbles, kibbles suggested, suggested two, two days, days vomiting, vomiting diarrhia, diarrhia stopped, stopped highly, highly recommend, recommend product, product thank, thank natural, natural balance, little westie benji, westie benji heartburn, benji heartburn diarrhea, heartburn diarrhea upset, diarrhea upset stomach, upset stomach issues, stomach issues quite, issues quite months, quite months throw, months throw almost, throw almost daily, almost daily took, daily took internest, took internest recommended, internest recommended feed, recommended feed kibble, feed kibble containing, kibble containing two, containing two ingredients, two ingredients proceed, ingredients proceed tests, proceed tests one, tests one kibbles, one kibbles suggested, kibbles suggested two, suggested two days, two days vomiting, days vomiting diarrhia, vomiting diarrhia stopped, diarrhia stopped highly, stopped highly recommend, highly recommend product, recommend product thank, product thank natural, thank natural balance)"
"List(i, put, 1, 2, teabags, of, easy, now, in, an, entire, teapot, let, it, steep, for, 20, 30, min, and, have, some, before, bed, no, matter, when, i, got, to, sleep, i, always, wake, up, 7, 8, hours, later, feeling, good, and, without, having, woken, up, in, the, middle, of, the, night, except, maybe, to, use, the, restroom, but, i, always, fall, back, asleep, instantly, its, also, great, anytime, during, the, day, to, just, calm, your, nerves, it, won, t, make, you, sleepy, this, is, easily, my, favorite, calming, tea, put 1, 1 2, 2 teabags, teabags easy, easy entire, entire teapot, teapot let, let steep, steep 20, 20 30, 30 min, min bed, bed matter, matter got, got sleep, sleep always, always wake, wake 7, 7 8, 8 hours, hours later, later feeling, feeling good, good without, without woken, woken middle, middle night, night except, except maybe, maybe use, use restroom, restroom always, always fall, fall back, back asleep, asleep instantly, instantly also, also great, great anytime, anytime day, day calm, calm nerves, nerves won, won make, make sleepy, sleepy easily, easily favorite, favorite calming, calming tea, put 1 2, 1 2 teabags, 2 teabags easy, teabags easy entire, easy entire teapot, entire teapot let, teapot let steep, let steep 20, steep 20 30, 20 30 min, 30 min bed, min bed matter, bed matter got, matter got sleep, got sleep always, sleep always wake, always wake 7, wake 7 8, 7 8 hours, 8 hours later, hours later feeling, later feeling good, feeling good without, good without woken, without woken middle, woken middle night, middle night except, night except maybe, except maybe use, maybe use restroom, use restroom always, restroom always fall, always fall back, fall back asleep, back asleep instantly, asleep instantly also, instantly also great, also great anytime, great anytime day, anytime day calm, day calm nerves, calm nerves won, nerves won make, won make sleepy, make sleepy easily, sleepy easily favorite, easily favorite calming, favorite calming tea)"
"List(this, is, a, chips, ahoy, flavor, called, ooey, gooey, chocofudge, so, it, should, alr

Similar to how we looked at the top tokens in our dataset, now we can use the `ngramDF` you created above to take a look at the most common n-grams (`n = 2` and `3`) in our dataset.

In [0]:
# Resulting top 25 ngrams
from pyspark.sql.functions import size, split, explode

ngramDist = (
    ngramDF.withColumn("indivNGrams", explode(col("ngrams")))
    .filter(size(split("indivNGrams", " ")) > 1)  # only keep ngrams with n>1
    .groupBy("indivNGrams")
    .count()
    .sort(col("count").desc())
)

display(ngramDist)

indivNGrams,count
gluten free,16435
amazon gp,16012
gp product,16007
amazon gp product,16005
k cups,15097
taste like,14921
highly recommend,14564
ve tried,14417
peanut butter,14057
dog food,13162


What do you notice about the frequent n-grams? Could they be important in text processing?

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>